# Solving the Learning Parity with Noise Problem using ML and MIO

### INTRODUCTION AND MOTIVATION
Imagine that Alice and Bob are two agents on a secret mission. Prior to every communication between them, they identify each other with the help of a shared secret key s. If Alice wants to identify herself to Bob, she needs to share her secret key with Bob so that Bob can verify that she is indeed Alice and not an imposter. However, by directly communicating the secret key s, an adversary may sniff the key and can later re-use it. Hence, this method of direct communication is not safe.
To tackle this, Bob can instead ask Alice a set of questions that Alice can answer using her secret key. The questions can ask Alice to reveal some bits at a time but not all of them at once. The questions asked by Bob and the answers given by Alice can still be overheard by the adversary, however, now the adversary will have to reverse engineer the key from the set of questions and answers. It is difficult but still tractable since the key can be recovered by solving a system of linear equations if the set of questions and answers is known.
However, Alice and Bob can still make the life of the adversary more difficult by adding slight random noise to the answers instead of providing the exact answers. The adversary will no longer be able to solve a linear system and the possibilities of the secret key will be exponential just by adding random noise. This is called Learning Parity with Noise (LPN) Problem.


### PROBLEM STATEMENT 
Imagine you are the adversary. Solve the LPN Problem for an n-dimensional secret key $s ∈{0,1}^n$ over m questions given by question matrix $A ∈{0,1}^(m*n)$ and m answers given answer vector $b ∈{0,1}^m$. The answers can have random noise $e ∈{0,1}^m$ that follows a Bernoulli distribution with noise parameter p < 0.5. The relationship between the questions, answers, noise, and secret key is given by:

$A*s+ e=b$

**Intuition**
Here, each row in matrix A represents a question about revealing specific bits of the secret key s. For example, over a 4-dimensional key s = [1 1 0 1], a question that asks about revealing the sum of 2nd and 4th bits will be represented by the vector [0 1 0 1]. The answer will be given by the XOR multiplication of matrix A with secret key s:
$[0 \ 1 \ 0  \ 1]*[1 \ 1 \ 0 \ 1]^T=0*1+1*1+0*0+1*1=0+1+0+1=0$
Hence, our answer is 0 which is represented by the corresponding row of vector b. We will have m such questions and answers. This process of revealing some but not all bits at a time is called HB Protocol .
Now, e is our noise vector that follows Bernoulli’s distribution with probability p. This means that we flip a biased coin that returns heads (or 1) with probability p and tails (or 0) with probability 1-p.  In our previous example, if the corresponding element of the error vector is 0, we get our final answer as 0 + 0 = 0 (which corresponds to the correct sum). However, if the corresponding element of the error vector is 1, our final answer is 0 + 1 = 1 (which corresponds to the incorrect sum). Since e is an m-dimensional vector, it will have approximately p*m elements as 1. 
We want to recover the secret key s, given Bob’s input questions and Alice’s noisy output answers. 


## DATASET

We will generate synthetic data for our problem using a fixed error probability (p < 0.5) and a secret key (s) of length n. Based on n and p, we will construct the dataset by simulating the question matrix A with dimensions m x n where m is given by : 


$m \le 4*n*(0.5-p)^(-2)  \ \  \ \ \ (A)$


We will also simulate a noise vector e that follows a Bernoulli distribution with error probability p. Finally, we will generate our answer vector using the following equation:

$(A*s + e)  \% 2 = b   \ \ \ \ (B)$


Here, we compute modulo 2 since A*s is XOR product and A*s + e is XOR addition. 

In [ ]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, Statistics, Random, Distributions, DataFrames, Combinatorics 

In [ ]:
function generateDistribution(n, p)
    m = floor(Int, n*p)
    a = ones(Int, m)
    b = zeros(Int, n-m)
    res = [a;b]
    return shuffle(res)
end

## Machine Learning

Given a fixed p, we will train an ML model for a synthetic input matrix A and output vector b in order to recover the secret key s. Note that this problem is a classification problem since each element of b is either 0 or 1. We will try various classification techniques such as Logistic Regression, SVM, Optimal Feature Selection, CART, OCT, OCT-H, Random Forest Classifier, and XGBoost Classifier. We will perform 3-fold cross-validation for hyperparameter tuning for sparsity, minbucket, depth, number of trees, and estimators. 
Evaluation

From our trained model, we will make predictions on test set $I^{(n*n)}$. We use the identity matrix of dimension n as our test set to recover each bit of the predicted secret key $\hat{s}$. From our previous example, the multiplication of a 4x4 identity matrix with secret key s, gives the secret key itself. Hence, we should get predictions for the identity set on the trained ML model.

To check our secret key $\hat{s}$, we will compute how much our predicted answer differs from the true answer:

$sum((A*\hat{s}+b)  \%  2)= error \ \ \  (C)$


Note that $(A*\hat{s}+b)  \%  2$ returns 1 if $A*\hat{s}$ is different from b, otherwise it returns 0.
We will also compute an error threshold to check if our error is small enough:
$\tau = p*m + \delta                            \ \ \     (D)$
We will assume that the key is recovered if error ≤ τ. Note that we need to do these gymnastics to evaluate our model since we don’t have the real key from Alice but only her noisy answers.


### GlmNET

In [ ]:
seed = 15095
p = 0.05
results = zeros(80, 6)
num_result = 1
for n=[4:2:32;]
    test = Matrix(I, n, n);
    m = floor(Int,4*n*((0.5-p)^(-2)))
    threshold = ceil(Int, m*p + sqrt(n*m))
    for ratio in [0, 0.125, 0.25, 0.375, 0.5]
        s = generateDistribution(n, ratio)
        count = 0
        time1 = 0
        for i=1:25
            A = bitrand((m,n))
            e = generateDistribution(m, p)
            b = (A*s + e).%2;

            # Train GLMNet
            lnr = IAI.GLMNetCVClassifier(random_seed=seed) 
            grid = IAI.GridSearch(lnr, n_folds=5)
            time1 = @elapsed IAI.fit!(grid, A, b)
            best_lnr = IAI.get_learner(grid)
            auc = round(IAI.score(best_lnr, A, b, positive_label=1, criterion=:auc), digits=2)
            s_pred = IAI.predict(best_lnr, test);

            # Verify Correctness
            error = sum((A*s_pred+b).%2)
            if error <= threshold
                count += 1
            end

            # Print Result
            println("For n=$n, ratio=$ratio, threshold=$threshold, key $s, i=$i, auc=$auc, error=$error")        
        end   
        result = [n, m, threshold, ratio, count, time1]
        results[num_result, :] = result
        num_result += 1
    end
end


### XGBoost


In [ ]:
seed = 15095
p = 0.05
results = zeros(80, 4)
num_result = 1
for n=[4:2:32;]
    test = Matrix(I, n, n);
    m = floor(Int,4*n*((0.5-p)^(-2)))
    threshold = ceil(Int, m*p + sqrt(n*m))
    for ratio in [0, 0.125, 0.25, 0.375, 0.5]
        s = generateDistribution(n, ratio)
        count = 0
        for i=1:25
            A = bitrand((m,n))
            e = generateDistribution(m, p)
            b = (A*s + e).%2;

            # Train 
            lnr = IAI.XGBoostClassifier(random_seed=seed) 
            grid = IAI.GridSearch(lnr, max_depth=setDepth(n), num_rounds=[20, 50, 100])
            time = @elapsed IAI.fit!(grid, A, b)
            best_lnr = IAI.get_learner(grid)
            auc = round(IAI.score(best_lnr, A, b, positive_label=1, criterion=:auc), digits=2)
            s_pred = IAI.predict(best_lnr, test);

            # Verify Correctness
            error = sum((A*s_pred+b).%2)
            if error <= threshold
                count += 1
            end

            # Print Result
            println("For n=$n, ratio=$ratio, threshold=$threshold, key $s, i=$i, auc=$auc, error=$error")        
        end   
        result = [n, m, threshold, ratio, count, time]
        results[num_result, :] = result
        num_result += 1
    end
end




### OCT

In [ ]:
seed = 15095
p = 0.05
results = zeros(80, 4)
num_result = 1
for n=[4:2:32;]
    test = Matrix(I, n, n);
    m = floor(Int,4*n*((0.5-p)^(-2)))
    threshold = ceil(Int, m*p + sqrt(n*m))
    for ratio in [0, 0.125, 0.25, 0.375, 0.5]
        s = generateDistribution(n, ratio)
        count = 0
        for i=1:25
            A = bitrand((m,n))
            e = generateDistribution(m, p)
            b = (A*s + e).%2;

            # Train 
            lnr = IAI.XGBoostClassifier(random_seed=seed) 
            grid = IAI.GridSearch(lnr, max_depth=setDepth(n), num_rounds=[20, 50, 100])
            time = @elapsed IAI.fit!(grid, A, b)
            best_lnr = IAI.get_learner(grid)
            auc = round(IAI.score(best_lnr, A, b, positive_label=1, criterion=:auc), digits=2)
            s_pred = IAI.predict(best_lnr, test);

            # Verify Correctness
            error = sum((A*s_pred+b).%2)
            if error <= threshold
                count += 1
            end

            # Print Result
            println("For n=$n, ratio=$ratio, threshold=$threshold, key $s, i=$i, auc=$auc, error=$error")        
        end   
        result = [n, m, threshold, ratio, count, time]
        results[num_result, :] = result
        num_result += 1
    end
end




### OCT-H


In [ ]:
seed = 15095
p = 0.05
results = zeros(80, 6)
num_result = 1
for n=[4:2:32;]
    test = Matrix(I, n, n);
    m = floor(Int,4*(n^1.5)*((0.5-p)^(-2)))
    threshold = ceil(Int, m*p + sqrt(n*m))
    for ratio in [0, 0.125, 0.25, 0.375, 0.5]
        s = generateDistribution(n, ratio)
        count = 0
        for i=1:25
            A = bitrand((m,n))
            e = generateDistribution(m, p)
            b = (A*s + e).%2;

            # Train OCT-H
            lnr = IAI.OptimalTreeClassifier(random_seed=seed, hyperplane_config=Dict("sparsity"=>:all)) 
            grid = IAI.GridSearch(lnr, max_depth=setDepth(n), minbucket=[3,4,5])
            IAI.fit!(grid, A, b)
            best_lnr = IAI.get_learner(grid)
            auc = round(IAI.score(best_lnr, A, b, positive_label=1, criterion=:auc), digits=2)
            s_pred = IAI.predict(best_lnr, test);

            # Verify Correctness
            error = sum((A*s_pred+b).%2)
            if error <= threshold
                count += 1
            end

            # Print Result
            println("For n=$n, ratio=$ratio, threshold=$threshold, key $s, i=$i, auc=$auc, error=$error")        
        end   
        result = [n, threshold, ratio, count]
        results[num_result, :] = result
        num_result += 1
    end
end




### Random forest

In [ ]:
seed = 15095
p = 0.05
results = zeros(80, 4)
num_result = 1
for n=[10:2:32;]
    test = Matrix(I, n, n);
    m = floor(Int,4*n*((0.5-p)^(-2)))
    threshold = ceil(Int, m*p + sqrt(n*m))
    for ratio in [0, 0.125, 0.25, 0.375, 0.5]
        s = generateDistribution(n, ratio)
        count = 0
        for i=1:25
            A = bitrand((m,n))
            e = generateDistribution(m, p)
            b = (A*s + e).%2;

            # Train RF
            lnr = IAI.RandomForestClassifier(random_seed=seed) 
            grid = IAI.GridSearch(lnr, max_depth=setDepth(n), minbucket=[3,4,5], num_trees=[10, 25, 50])
            time = @elapsed IAI.fit!(grid, A, b)
            best_lnr = IAI.get_learner(grid)
            auc = round(IAI.score(best_lnr, A, b, positive_label=1, criterion=:auc), digits=2)
            s_pred = IAI.predict(best_lnr, test);

            # Verify Correctness
            error = sum((A*s_pred+b).%2)
            if error <= threshold
                count += 1
            end

            # Print Result
            println("For n=$n, ratio=$ratio, threshold=$threshold, key $s, i=$i, auc=$auc, error=$error")        
        end   
        result = [n, m, threshold, ratio, count, time]
        results[num_result, :] = result
        num_result += 1
    end
end




## Mixed Integer Optimization

Given a fixed p, we will solve an MIO problem for a synthetic input matrix A and output vector b in order to recover the secret key s. Note that this problem will be a mixed-integer problem since each element of key s is either 0 or 1 and b is the XOR product of A and s plus some error term. The advantage of this approach will be that it will always recover the key for reasonable number of samples, unlike the ML models.

Our objective will be to minimize the difference between A*s and b since this difference corresponds to our Bernoulli error p:

$$min_{s\in\{ 0,1 \}^n }     \sum_{j=1}^m |a_j*s_j-b_j |$$

Now since everything is a XOR operation here, we need to model the constraints of XOR multiplication and XOR addition. To be able to do that, for every multiplication and addition operation, we need to compute the result of the operation modulus 2. Let $h_j$ be the remainder when $a_j*s_j$ is divided by 2. Our objective now becomes:

$$min_{s\in\{ 0,1 \}^n }     \sum_{j=1}^m |h_j-b_j |$$
$$s.t.      \sum_{i=1}^n A_{ji}*s_i =2y_j+h_j                \ \   j\in m$$
$$h,y \ge 0$$
$$h_j<2                                               j\in m$$     
$$s\in\{ 0,1 \}^n, h \in R^m, y \in Z^m$$



We’re almost there, now let’s linearize it and get the final MIO formulation!



**MIO Formulation**: 


min_{s,h,z,y} \sum_{j=1}^m z_j 
$$s.t.      \sum_{i=1}^n A_{ji}*s_i =2y_j+h_j                \ \   j\in m$$
$$h_j -b_j<_j                                               j\in m$$     
$$-h_j + b_j<_j                                               j\in m$$     
$$h,y,z \ge 0$$
$$h_j<2                                               j\in m$$     
$$s\in\{ 0,1 \}^n, h,z \in R^m, y \in Z^m$$



In [ ]:
function MIO(A,b,n,m)
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)
    set_optimizer_attribute(model, "TimeLimit", 1800)  # limit 30 minutes 
    epsilon = 0.0001

    # Add variables
    @variable(model, s[i=1:n], Bin)
    @variable(model, h[j = 1:m] >= 0)
    @variable(model, z[j = 1:m] >= 0)
    @variable(model, y[j = 1:m] >= 0, Int)

    @objective(model, Min, sum(z[j] for j = 1:m))

    @constraint(model, [j=1:m] ,   h[j] - b[j]  <= z[j])
    @constraint(model, [j=1:m] , - h[j] + b[j]  <= z[j])
    @constraint(model, [j=1:m] , sum(A[j,i]*s[i] for i = 1:n)  == 2*y[j] + h[j])
    @constraint(model, [j=1:m] ,  h[j] <= 2 - epsilon)

    time1 = @elapsed optimize!(model)
    return value.(s), time1
end 

In [ ]:
seed = 15095
p = 0.1
results = zeros(5000, 6)
num_result = 1
for n=[4:2:20;]
    test = Matrix(I, n, n);
    M = floor(Int,4*n*((0.5-p)^(-2)) )
    for ratio in [0, 0.25, 0.5, 0.75, 1]
        s = generateDistribution(n, ratio)

        for m = [Int(trunc(M/50)):Int(trunc(M/50)):Int(trunc(M/10));] 
            count = 0
            threshold = ceil(Int, m*p + sqrt(n*m))
            A = bitrand((m,n))
            e = rand(Binomial(1,p), m)
            b = (A*s + e).%2;

            # Solve MIO
            s_pred, time1 =  MIO(A,b,n,m);

            # Verify Correctness
            error = sum((A*s_pred+b).%2)
            if error < threshold
                count += 1
            end
        
            # Print Result
            println("For n=$n, ratio=$ratio, threshold=$threshold, key $s, m=$m,  error=$error")          
            result = [n, m, threshold, ratio, count, time1] # count 1 recovered
            results[num_result, :] = result
            num_result += 1
        end
    end
end

## Brute Force Algorithm
We will also use the Brute-Force algorithm to find the secret key s by iterating over all possible keys until a given key satisfies the threshold criteria in (D). The solution from this approach will serve as a benchmark both in terms of correctness as well as time complexity.


In [ ]:
function brute_force(s,t,A,b)
    n = length(s)    
    tic = now()
    # Generate new s
    for i in 0:n
        s_f = vcat(repeat([1],i), repeat([0],n-i))
        for s_try in multiset_permutations(s_f,length(s_f)) 
            if sum((A*s_try+b).%2) <= t
                return s_try
            end

            toc = now()
            if toc-tic > Minute(30)
                return s_try
            end
        end 
    end 

end 


In [ ]:
seed = 15095
p = 0.1
results = zeros(5000, 6)
num_result = 1
for n=[4:2:20;]
#for n=[4:2:4;]
    test = Matrix(I, n, n);
    M = floor(Int,4*n*((0.5-p)^(-2)) )
    for ratio in [0, 0.25, 0.5, 0.75, 1]
        s = generateDistribution(n, ratio)

        for m = [Int(trunc(M/50)):Int(trunc(M/50)):Int(trunc(M/10));] 
            count = 0
            threshold = ceil(Int, m*p)
            A = bitrand((m,n))
            e = generateDistribution(m, p)
            b = (A*s + e).%2;

            # Solve Brute force
            s_pred = brute_force(s,threshold,A,b);
            time1 =  @elapsed brute_force(s,threshold,A,b);

            # Verify Correctness
            error = sum((A*s_pred+b).%2)
            if error <= threshold
                count += 1
            end
            recover = (s_pred == s)
            # Print Result
            println("For n=$n, ratio=$ratio, threshold=$threshold, key $s, m=$m, recover=$recover, error=$error")          
            result = [n, m, threshold, ratio, count, time1] # count 1 recovered
            results[num_result, :] = result
            num_result += 1
        end
    end
end